In [9]:
#pip install rasterstats

In [5]:
#pip install matplotlib

In [3]:
pip install scikit-learn



     ---------------------------------------- 8.4/8.4 MB 289.3 kB/s eta 0:00:00
     ------------------------------------ 298.0/298.0 kB 227.5 kB/s eta 0:00:00
     -------------------------------------- 42.5/42.5 MB 176.3 kB/s eta 0:00:00


In [4]:
import pandas as pd
import rasterio
from rasterstats import zonal_stats
from osgeo import gdal
import geopandas as gpd
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon
from pyproj import CRS
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
landcover = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/Isaias_Nj_Lat_Long_500.csv")
landcover.head()

,Unnamed: 0,ID,latitude,longitude,value
0,0,0,41.415450,-75.846241,128
1,1,1,41.410565,-75.846241,128
2,2,2,41.405680,-75.846241,128
3,3,3,41.400794,-75.846241,128
4,4,4,41.395909,-75.846241,128


In [6]:
wind = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Isaias_NJ_speed_backwinds_akshay.csv")
wind.head()

,ID,Unnamed: 0,latitude,longitude,value,Distance,Vmax,Rmax
0,0,0,41.415450,-75.846241,128,325.294594,30.030877,120.546311
1,1,1,41.410565,-75.846241,128,325.583200,30.100979,120.546311
2,2,2,41.405680,-75.846241,128,325.871899,30.102396,120.546311
3,3,3,41.400794,-75.846241,128,326.160691,30.173096,120.546311
4,4,4,41.395909,-75.846241,128,326.449577,30.174527,120.546311


In [7]:
print(landcover.shape)
print(wind.shape)

(131364, 5)
(131364, 8)


- get zonal histogram for landcover data 

In [8]:
#check resolution of raster
with rasterio.open("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/NJ_All Layers_4326.tif") as src:
    # Print the bounding box coordinates
    print('Bounding box coordinates:', src.bounds)

    # Print the CRS
    print('CRS:', src.crs)

    #print resolution of raster
    print('Resolution:', src.res)

Bounding box coordinates: BoundingBox(left=-75.84624120900405, bottom=38.81163163562519, right=-73.54287375628081, top=41.41545006869191)
CRS: EPSG:4326
Resolution: (0.00027018973052471917, 0.00027018973052471917)


In [9]:
landcover.head()

,Unnamed: 0,ID,latitude,longitude,value
0,0,0,41.415450,-75.846241,128
1,1,1,41.410565,-75.846241,128
2,2,2,41.405680,-75.846241,128
3,3,3,41.400794,-75.846241,128
4,4,4,41.395909,-75.846241,128


In [10]:
zonal = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/zonal_500.csv")
zonal.head()

,id,left,top,right,bottom,HISTO_0,HISTO_1,HISTO_5,HISTO_6,HISTO_8,HISTO_10,HISTO_14,HISTO_15,HISTO_16,HISTO_17,HISTO_18,HISTO_NODATA
0,1,2054340,-81750,2054840,-82250,0,0,0,0,0,0,0,0,0,0,0,289
1,2,2054340,-82250,2054840,-82750,0,0,0,0,0,0,0,0,0,0,0,272
2,3,2054340,-82750,2054840,-83250,0,0,0,0,0,0,0,0,0,0,0,289
3,4,2054340,-83250,2054840,-83750,0,0,0,0,0,0,0,0,0,0,0,289
4,5,2054340,-83750,2054840,-84250,0,0,0,0,0,0,0,0,0,0,0,272


In [11]:
zonal.iloc[:5,:5]

,id,left,top,right,bottom
0,1,2054340,-81750,2054840,-82250
1,2,2054340,-82250,2054840,-82750
2,3,2054340,-82750,2054840,-83250
3,4,2054340,-83250,2054840,-83750
4,5,2054340,-83750,2054840,-84250


In [12]:
zonal.shape

(131610, 17)

In [13]:
#check resolution of raster
with rasterio.open("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/NJ_All Layers_4326_500_right_cropped_adjusted.tif") as src:
    # Print the bounding box coordinates
    print('Bounding box coordinates:', src.bounds)

    # Print the CRS
    print('CRS:', src.crs)

    #print resolution of raster
    print('Resolution:', src.res)

Bounding box coordinates: BoundingBox(left=2054340.0, bottom=-349250.0, right=2177340.0, top=-81750.0)
CRS: PROJCS["WGS_1984_Lambert_Azimuthal_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",45],PARAMETER["longitude_of_center",-100],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Resolution: (500.0, 500.0)


In [14]:
nightlights_zonal = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/Temp/zonal_nightlights_day_01.csv")
nightlights_zonal.head()

,id,left,top,right,bottom,_count,_sum,_mean
0,1,2054340,-81750,2054840,-82250,NaN,NaN,NaN
1,2,2054340,-82250,2054840,-82750,NaN,NaN,NaN
2,3,2054340,-82750,2054840,-83250,NaN,NaN,NaN
3,4,2054340,-83250,2054840,-83750,NaN,NaN,NaN
4,5,2054340,-83750,2054840,-84250,NaN,NaN,NaN


In [15]:
nightlights_mean = nightlights_zonal[["id","_mean"]]
nightlights_mean.head()

,id,_mean
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN


In [16]:
nightlights_mean['_mean'].notna().sum()

69750

In [17]:
nightlights_mean=nightlights_mean.dropna()
nightlights_mean.shape

(69750, 2)

In [18]:
zonal.head()

,id,left,top,right,bottom,HISTO_0,HISTO_1,HISTO_5,HISTO_6,HISTO_8,HISTO_10,HISTO_14,HISTO_15,HISTO_16,HISTO_17,HISTO_18,HISTO_NODATA
0,1,2054340,-81750,2054840,-82250,0,0,0,0,0,0,0,0,0,0,0,289
1,2,2054340,-82250,2054840,-82750,0,0,0,0,0,0,0,0,0,0,0,272
2,3,2054340,-82750,2054840,-83250,0,0,0,0,0,0,0,0,0,0,0,289
3,4,2054340,-83250,2054840,-83750,0,0,0,0,0,0,0,0,0,0,0,289
4,5,2054340,-83750,2054840,-84250,0,0,0,0,0,0,0,0,0,0,0,272


In [19]:
regression_df = zonal.merge(wind,how="left", left_on="id",right_on="ID").fillna(0)
regression_df = regression_df.merge(nightlights_mean,how="inner",on="id")
regression_df.head()

,id,left,top,right,bottom,HISTO_0,HISTO_1,HISTO_5,HISTO_6,HISTO_8,HISTO_10,HISTO_14,HISTO_15,HISTO_16,HISTO_17,HISTO_18,HISTO_NODATA,ID,Unnamed: 0,latitude,longitude,value,Distance,Vmax,Rmax,_mean
0,99,2054340,-130750,2054840,-131250,0,0,1,9,0,0,0,0,0,1,40,238,99.0,99.0,40.931814,-75.846241,5.0,354.285334,34.452763,120.546311,0.800000
1,100,2054340,-131250,2054840,-131750,0,0,120,12,0,0,0,0,0,24,65,68,100.0,100.0,40.926929,-75.846241,17.0,354.582134,34.455151,120.546311,0.843006
2,101,2054340,-131750,2054840,-132250,0,0,183,0,0,0,0,0,0,20,37,32,101.0,101.0,40.922044,-75.846241,18.0,354.879006,34.457554,120.546311,0.900000
3,102,2054340,-132250,2054840,-132750,0,0,114,0,0,0,0,0,0,38,39,98,102.0,102.0,40.917158,-75.846241,17.0,355.175950,34.459970,120.546311,1.244677
4,103,2054340,-132750,2054840,-133250,0,0,92,23,0,0,0,0,0,39,37,98,103.0,103.0,40.912273,-75.846241,17.0,355.472967,34.572871,120.546311,1.050000


In [20]:
regression_df.shape

(69750, 26)

In [21]:
regression_df.columns

Index(['id', 'left', 'top', 'right', 'bottom', 'HISTO_0', 'HISTO_1', 'HISTO_5',
       'HISTO_6', 'HISTO_8', 'HISTO_10', 'HISTO_14', 'HISTO_15', 'HISTO_16',
       'HISTO_17', 'HISTO_18', 'HISTO_NODATA', 'ID', 'Unnamed: 0', 'latitude',
       'longitude', 'value', 'Distance', 'Vmax', 'Rmax', '_mean'],
      dtype='object')

In [22]:
regression_df = regression_df.drop(['id', 'left', 'top', 'right', 'bottom','ID', 'Unnamed: 0', 'latitude',
       'longitude', 'value'], axis=1)
regression_df.shape

(69750, 16)

In [23]:
regression_df.columns

Index(['HISTO_0', 'HISTO_1', 'HISTO_5', 'HISTO_6', 'HISTO_8', 'HISTO_10',
       'HISTO_14', 'HISTO_15', 'HISTO_16', 'HISTO_17', 'HISTO_18',
       'HISTO_NODATA', 'Distance', 'Vmax', 'Rmax', '_mean'],
      dtype='object')

In [25]:
df = regression_df.copy()

# Separate independent and dependent variables
X = df.drop('_mean', axis=1)  # Independent variables
y = df['_mean']  # Dependent variable

# Standardize independent variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create and fit the regression model
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

# Print the model coefficients
print("Model Coefficients:")
for feature, coef in zip(X.columns, reg_model.coef_):
    print(feature, ":", coef)

# Predict on the test set
y_pred = reg_model.predict(X_test)

# Calculate R2 score on the test set
r2 = r2_score(y_test, y_pred)

# Print the R2 score
print("R2 score on the test set:", r2)

Model Coefficients:
HISTO_0 : -0.30662816612950566
HISTO_1 : -1.3299855929454527
HISTO_5 : -2.7531473806983806
HISTO_6 : -1.3575605034605127
HISTO_8 : -0.3815034543436345
HISTO_10 : -0.19169559359328092
HISTO_14 : -1.9187636766610694
HISTO_15 : -2.3943268723006756
HISTO_16 : -0.2653341001037971
HISTO_17 : 5.6957047637316185
HISTO_18 : -0.20036713932923386
HISTO_NODATA : 0.061567445928782716
Distance : -0.36077544082759083
Vmax : 0.4662419852352969
Rmax : 0.029634255786064684
R2 score on the test set: 0.3161069670085993
